# 3 - PDF RAG con HuggingFace y LangChain

<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/rag_1.webp" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Definiciones-y-dependencias." data-toc-modified-id="1.-Definiciones-y-dependencias.-0">1. Definiciones y dependencias.</a></span></li><li><span><a href="#2.-Carga-de-archivos-PDF." data-toc-modified-id="2.-Carga-de-archivos-PDF.-1">2. Carga de archivos PDF.</a></span><ul class="toc-item"><li><span><a href="#Document-Loaders-de-LangChain" data-toc-modified-id="Document-Loaders-de-LangChain-1.1"><a href="https://python.langchain.com/docs/modules/data_connection/document_loaders/" rel="nofollow" target="_blank">Document Loaders de LangChain</a></a></span></li></ul></li><li><span><a href="#3.-Troceando-los-documentos-(Chunks)" data-toc-modified-id="3.-Troceando-los-documentos-(Chunks)-2">3. Troceando los documentos (Chunks)</a></span></li><li><span><a href="#4.-Preceso-de-Embedding" data-toc-modified-id="4.-Preceso-de-Embedding-3">4. Preceso de Embedding</a></span></li><li><span><a href="#5.-Guardado-de-embeddings-en-Chroma-DB" data-toc-modified-id="5.-Guardado-de-embeddings-en-Chroma-DB-4">5. Guardado de embeddings en Chroma DB</a></span></li><li><span><a href="#6.-Búsqueda-de-documentos-relevantes-en-Chroma" data-toc-modified-id="6.-Búsqueda-de-documentos-relevantes-en-Chroma-5">6. Búsqueda de documentos relevantes en Chroma</a></span></li><li><span><a href="#7.-Plantillas-de-instrucciones-(Prompt-Templates)" data-toc-modified-id="7.-Plantillas-de-instrucciones-(Prompt-Templates)-6">7. Plantillas de instrucciones (<a href="https://python.langchain.com/docs/modules/model_io/prompts/quick_start/" rel="nofollow" target="_blank">Prompt Templates</a>)</a></span></li><li><span><a href="#8.-Modelo-LLM" data-toc-modified-id="8.-Modelo-LLM-7">8. Modelo LLM</a></span></li><li><span><a href="#9.-Construcción-de-la-cadena" data-toc-modified-id="9.-Construcción-de-la-cadena-8">9. Construcción de la cadena</a></span></li><li><span><a href="#10.-Resumen-paso-a-paso" data-toc-modified-id="10.-Resumen-paso-a-paso-9">10. Resumen paso a paso</a></span></li></ul></div>

## 1. Definiciones y dependencias. 

**Dependencias necesarias para este workshop:**


```bash
pip install langchain
pip install langchain-chroma
pip install pypdf
pip install transformers
pip install sentence-transformers
pip install transformers[sentencepiece]

```

El propósito de este workshop es crear un RAG (Retrieval Augmented Generation) con [LangChain](https://www.langchain.com/) y un modelo de código abierto desde [HuggingFace](https://huggingface.co/).

**¿Qué es un LLM?**


Un LLM, o "Large Language Model" (Gran Modelo de Lenguaje), es un tipo de modelo de inteligencia artificial diseñado para entender, generar y trabajar con lenguaje humano a gran escala. Estos modelos se entrenan utilizando grandes cantidades de datos de texto para aprender patrones, estructuras del lenguaje y relaciones contextuales.

Características principales de los LLM:

1. Capacidad de Generación de Texto: Los LLMs son capaces de generar texto coherente y contextualmente relevante que puede imitar el estilo y la estructura del lenguaje humano. Esto los hace útiles para tareas como la escritura automática de artículos, generación de respuestas en chatbots, etc...

2. Comprensión del Contexto: Gracias a su entrenamiento con grandes cantidades de texto, los LLMs tienen una notable capacidad para entender el contexto de las consultas que reciben, lo que les permite ofrecer respuestas más precisas y relevantes.

3. Aplicaciones Multilingües: Algunos LLMs están entrenados en múltiples idiomas, lo que les permite operar en diferentes lenguajes y realizar tareas como traducción automática o asistencia multilingüe.

4. Aprendizaje Continuo: Aunque los LLMs se entrenan en un conjunto estático de datos, algunos modelos están diseñados para continuar aprendiendo a partir de nuevas interacciones, lo que mejora su rendimiento y adaptabilidad con el tiempo.

5. Interpretación de Sentimiento y Semántica: Pueden analizar y entender sentimientos, opiniones y matices semánticos en el texto, lo que es crucial para aplicaciones como análisis de sentimiento, soporte al cliente y monitorización de redes sociales.

Ejemplos de LLM:

+ GPT (Generative Pre-trained Transformer) de OpenAI: Es uno de los modelos de lenguaje más conocidos y avanzados, usado ampliamente en aplicaciones comerciales y de investigación por su capacidad para generar texto altamente contextual y creativo.

+ BERT (Bidirectional Encoder Representations from Transformers) de Google: Se utiliza principalmente para mejorar la comprensión del lenguaje en buscadores y para mejorar la precisión de las respuestas en aplicaciones de inteligencia artificial.


Cuando se habla de los parámetros de un LLM nos estamos refiriendo a los valores internos que el modelo utiliza para realizar predicciones y generar texto basado en el lenguaje humano. Estos parámetros son esenciales para que el modelo funcione correctamente y son ajustados durante el proceso de entrenamiento. Los parámetros son en su mayoría pesos sinápticos. Estos pesos determinan la fuerza de la conexión entre las neuronas en diferentes capas del modelo. Durante el entrenamiento, estos pesos se ajustan para minimizar el error en la predicción del modelo. Junto con los pesos, los biases (sesgos) son otro tipo de parámetros que se añaden a las sumas ponderadas en las neuronas para ayudar al modelo a ajustarse mejor a los datos, son el equivalente a la ordenada en el origen. Los biases permiten que el modelo opere eficazmente cuando todas las entradas son cero.

Los LLMs son conocidos por tener un número extremadamente alto de parámetros. Por ejemplo, GPT-3 de OpenAI tiene 175 mil millones de parámetros, lo que lo hace uno de los modelos de lenguaje más grandes y poderosos disponibles. Estos parámetros permiten al modelo capturar una gran cantidad de matices lingüísticos y contextuales.

**¿Qué es un RAG?**

La generación mejorada por recuperación (Retrieval Augmented Generation - RAG) es el proceso de optimización de un modelo lingüístico de gran tamaño (Large Language Model - LLM), de modo que conozca datos proporcionados por el usuario, que no existan en los datos de entrenamiento del modelo, antes de generar una respuesta. 

Los LLM se entrenan con grandes volúmenes de datos y usan miles de millones de parámetros para generar resultados originales en tareas como responder preguntas, traducir idiomas y completar frases. Un RAG extiende las ya poderosas capacidades de los LLM a dominios específicos o a la base de conocimientos interna de una organización, todo ello sin la necesidad de volver a entrenar el modelo. Se trata de un método rentable para mejorar los resultados de los LLM de modo que sigan siendo relevantes, precisos y útiles en diversos contextos.

Un RAG aporta varios beneficios directos en el desarrollo de una herramienta de inteligencia artificial:

+ Rentabilidad de la implementación. El desarrollo de IAs comienza normalmente con un modelo básico. Los modelos fundacionales (Foundational Models - FM) son LLMs accesibles por API entrenados en un amplio espectro de datos generalizados y sin etiquetar. Los costos computacionales y financieros de volver a entrenar a los FM para obtener información específica de la organización o del dominio son muy elevados. Un RAG es un enfoque más rentable para introducir nuevos datos en el LLM.


+ Información actualizada. Incluso si el LLM está entrenado con los datos adecuados para las necesidades de la compañía, es complicado mantener la relevancia del modelo. Un RAG permite a los desarrolladores proporcionar las últimas investigaciones, estadísticas o noticias a los modelos generativos. Se puede usar un RAG para conectar el LLM directamente a las redes sociales en vivo, sitios de noticias u otras fuentes de información que se actualizan con frecuencia. De esta manera, un LLM puede proporcionar la información más reciente.


+ Confianza. Al darle al LLM datos propios, se conoce perfectamente la fuente de datos además de evitar la alucinación del LLM.


+ Mayor control. El RAG permite a los desarrolladores de inteligencia artificial cambiar las fuentes de información para adaptarse a los requisitos cambiantes o a los usos múltiples de la compañía. Además pueden restringir la recuperación de información confidencial a diferentes niveles de autorización y garantizar que el LLM genere las respuestas adecuadas.


El esquema básico de un RAG es como sigue:

<br>

![rag](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/rag.png)

<br>

1. Los documentos de la compañía se pasan por un modelo de incrustación (Embedding Model) para ser guardados en forma de vectores en una base de datos.

2. La consulta realizada por el usuario pasa por el mismo modelo para convertir dicho texto en vectores.

3. Se buscan en la base de datos los vectores más parecidos a la consulta y se extraen los vectores más relevantes.

4. Los documentos más relevantes extraídos y la consulta se introducen en el LLM para generar la respuesta más adecuada.


![langchain](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/langchain.jpeg)

**¿Qué es LangChain?**


LangChain es un framework de código abierto para crear aplicaciones basadas en LLM. LangChain proporciona herramientas y abstracciones para mejorar la personalización, precisión y relevancia de la información que generan los modelos. Por ejemplo, los desarrolladores pueden usar los componentes de LangChain para crear nuevas cadenas de peticiones o personalizar las plantillas existentes. LangChain también incluye componentes que permiten a los LLM acceder a nuevos conjuntos de datos sin necesidad de repetir el entrenamiento.

![huggingface](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/huggingface.png)

**¿Qué es HuggingFace?**

HuggingFace es una empresa de tecnología que se especializa en inteligencia artificial (IA), particularmente en el área de procesamiento del lenguaje natural (NLP). Es conocida por desarrollar y mantener la biblioteca "Transformers", que proporciona modelos de IA pre-entrenados y herramientas para facilitar la construcción de aplicaciones relacionadas con el lenguaje, como la traducción automática, el análisis de sentimientos y la generación de texto. La compañía también contribuye a la investigación en IA y fomenta una comunidad activa de desarrolladores y investigadores que colaboran en proyectos de código abierto. Además, HuggingFace opera una plataforma en línea donde los usuarios pueden experimentar con diferentes modelos de IA, compartir sus propios modelos y colaborar en proyectos de IA. Esta plataforma también facilita la implementación y el uso de modelos de IA en diversas aplicaciones prácticas.

## 2. Carga de archivos PDF.


![pdf](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/pdf.png)

Formato de Documento Portátil (Portable Document Format - PDF), estandarizado como ISO 32000, es un formato de archivo desarrollado por Adobe en 1992 para presentar documentos, incluyendo el formato de texto e imágenes, de manera independiente del software de aplicación, hardware y sistemas operativos. El objetivo va a ser cargar archivos PDF para poder hacer preguntas a un modelo de lenguage (Large Language Model - LLM) al respecto de ellos. A esto se lo conoce como modelos de respuesta a preguntas (question answering models). Los modelos QA son sistemas de inteligencia artificial diseñados para responder preguntas formuladas en lenguaje natural. Estos modelos son una parte fundamental del procesamiento del lenguaje natural y se utilizan en una amplia gama de aplicaciones, desde asistentes virtuales hasta herramientas de búsqueda y análisis de información. El proceso de estos modelos es como sigue:

1. Comprensión de texto: Estos modelos generalmente están entrenados para entender y procesar texto escrito en un lenguaje humano. Pueden analizar grandes volúmenes de texto y extraer información relevante.

2. Búsqueda de información: Identifican partes del texto que son relevantes para la pregunta. Esto puede implicar la extracción directa de la respuesta de un texto específico o la inferencia de una respuesta basada en múltiples fragmentos de texto.

3. Generación de respuestas: Una vez que se identifica la información relevante, el modelo genera una respuesta en lenguaje natural.

Existen dos tipos de modelos QA:

+ Basados en recuperación (retrieval-based): Estos modelos buscan en una base de datos predefinida o en un conjunto de documentos para encontrar la respuesta, generalmente devolviendo el fragmento de texto más relevante. Este será el tipo de modelo QA que vamos a utilizar.

+ Basados en generación (generative-based): Estos modelos pueden generar respuestas nuevas que no están explícitamente presentes en los textos de entrada, basándose en el conocimiento adquirido durante su entrenamiento.


Los datos los hemos descargado de [Indra](https://www.indracompany.com/es/accionistas/memoria-cuentas-anuales/), donde están reflejados los resultados de la compañía anualmente de manera pública, y los hemos guardado en la carpeta `pdfs`.

In [ ]:
import os   # libreria del sistema operativo

In [ ]:
# lista de archivos en la carpeta pdfs

os.listdir('pdfs')  

### [Document Loaders de LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/)


Los "document loaders" de LangChain son componentes diseñados para cargar y procesar documentos dentro de la plataforma LangChain. "Los document loaders" en LangChain se utilizan para:

1. Cargar documentos: Pueden cargar documentos desde diversas fuentes, como archivos locales, bases de datos o servicios en la nube.

2. Procesar y preparar documentos: Facilitan la extracción y preparación de texto de los documentos para su posterior análisis y procesamiento por parte de modelos de lenguaje.

3. Indexación: Ayudan en la indexación de los documentos para facilitar la búsqueda y recuperación rápida de información relevante, lo que es crucial para tareas como la respuesta a preguntas basada en documentos.


Usaremos el `PyPDFLoader` de LangChain, el cual nos permite cargar y extraer texto de archivos PDF para que puedan ser procesados por modelos de lenguaje o utilizados en aplicaciones de respuesta a preguntas. Nos permite extraer texto de documentos PDF, lo cual es esencial para procesar este tipo de documentos que son comunes en muchos contextos profesionales y académicos. Esto incluye la capacidad de manejar múltiples páginas y extraer el texto de manera eficiente. Además incluye funciones para preprocesar el texto extraído. Esto puede implicar la eliminación de encabezados y pies de página, corrección de errores de OCR, y normalización del texto para prepararlo para análisis posteriores. Puede configurarse para trabajar con diferentes rutas de documentos y ajustar su comportamiento según las necesidades del usuario, como especificar rangos de páginas o seleccionar modos de extracción de texto específicos. Para poder usar este "loader", es necesario instalar la librería `pypdf`, comentado en las depedencias del workshop.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# carga de archivo PDF página a página

loader = PyPDFLoader('pdfs/memoria_consolidada_2022.pdf')

paginas = loader.load()

In [ ]:
len(paginas)

In [ ]:
paginas[10]

Existen más "loaders" de documentos PDF en LangChain. En particular existen dos que son capaces de reconocer la estructura tabular dentro del PDF. El primero es `UnstructuredPDFLoader`, basado en la librería [unstructured](https://github.com/Unstructured-IO/unstructured), sin embargo aún está en desarrollo y falla con frecuencia. El segundo es `AmazonTextractPDFLoader`, basado en el servicio de AWS Textract, un sistema intregado de Amazon para el reconocimiento de archivos de texto o PDFs y va más allá del simple reconocimiento óptico de caracteres (OCR). Para usarlo es necesario cuenta de desarrollador en AWS y el uso de `S3`, pudiendo conectarnos a través de la librería `boto3` de Python,

Veíamos que teniamos varios documentos PDF en nuestra carpeta, uno correspondiente al año 2022 y otro al año 2023. Con `PyPDFLoader` podemos cargar toda la carpeta a la vez. Veamos como:

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [ ]:
# carga de todos los archivos PDF página a página

loader = PyPDFDirectoryLoader('pdfs/')

paginas = loader.load()

In [ ]:
len(paginas)

## 3. Troceando los documentos (Chunks)

Ahora tenemos 828 páginas de ambos documentos PDF. Para un uso óptimo del contexto del LLM que vayamos a usar, vamos a trocear (chunk) las páginas del PDF. Podríamos usar cualquiera de los [text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/) que tiene LangChain, dado que nuestros documentos PDF ahora están en formato texto. Pero el propio `PyPDFLoader` tiene un método para hacer esto directamente, en vez de usar el método `load()` se usa el método `load_and_split()`.

![chunks](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/chunks.png)

In [ ]:
%%time

# carga de todos los archivos PDF y realiza los chunks

loader = PyPDFDirectoryLoader('pdfs/')

documentos = loader.load_and_split()

len(documentos)

In [ ]:
documentos[10]

## 4. Preceso de Embedding

Ahora tenemos 1048 trozos (chunks) listos para el embedding. Los embeddings permiten que los modelos de aprendizaje automático trabajen con datos complejos, como texto, de una manera más efectiva al transformarlos en formatos que son más fáciles de analizar y procesar. Se trata de vectorizar el texto de tal manera que podamos extraer los textos más relevantes al respecto de nuestra pregunta. Usaremos los modelos de embedding desde HuggingFace, además LangChain ya tiene incorporado la conexión a dichos modelos.

![embedding](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/embedding.png)


Desde `sentence-transformers` usaremos el modelo [all-roberta-large-v1](https://huggingface.co/sentence-transformers/all-roberta-large-v1), el cual convierte frases y párrafos a vectores densos de 1024 elementos. Vector denso se refiere a que no existen elementos con valor 0, sino que todos los 1024 elementos son distintos de 0. Dichos vectores serán después usados para la búsqueda semántica dentro del RAG, es decir, los usaremos para extraer esos textos más relevantes respectos a la consulta que hagamos.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# inicializamos el modelo de embedding con Roberta

vectorizador = HuggingFaceEmbeddings(model_name='sentence-transformers/all-roberta-large-v1')

In [ ]:
# realizamos una prueba para comprobar que funciona correctamente

vector = vectorizador.embed_query(documentos[10].page_content)

vector[:3]

In [ ]:
len(vector)

## 5. Guardado de embeddings en Chroma DB

![chroma](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/chroma.jpeg)

Chroma es una base de datos vectorial. Como tal, su objetivo es que seamos capaces de guardar vectores, los embeddings que creamos desde el texto, para después dotar de dicha información al LLM o simplemente para tenerlos guardados.

A nivel general, la forma de usar Chroma es la siguiente:

1. Crear nuestra colección, lo que es el equivalente a una tabla en una base de datos relacional. En este proceso, deberemos indicar qué modelo debe usar Chroma para convertir los textos en embeddings. En nuestro caso usaremos el modelo de embedding que hemos cargado anteriormente.

2. Enviar a Chroma un texto que queramos que guarde, junto con los metadatos que queramos para el filtrado del texto. Cuando Chroma reciba el texto se encargará de convertirlo a embedding con el modelo que le damos.

3. Consultar Chroma enviando un texto o un embedding, recibiremos los k documentos más parecidos, siendo k un parámetro de la consulta. Además, podremos filtrar la consulta en base a metadatos para que únicamente se ejecute sobre los documentos que cumplan una serie de criterios.


Todo este proceso se realiza desde el `vectorstores` que tiene LangChain, simplemente dándole los documentos, el modelo de embedding y la ruta de guardado de la base de datos al objeto `Chroma` incorporado.

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
%%time

# guardado en disco

chroma_db = Chroma.from_documents(documentos,                    # documentos de texto
                                  vectorizador,                  # modelo de embedding
                                  persist_directory='save_db'    # ruta de guardado
                                 )

## 6. Búsqueda de documentos relevantes en Chroma

Vamos a cargar la base de datos de Chroma desde los archivos locales y realizar una búsqueda para recuperar los documentos más relevantes según la consulta realizada. Iniciamos la carga de la base de datos con la ruta a los archivos que hemos guardado anteriormente y la función de embedding, el vectorizador que ya hemos definido.

In [ ]:
# carga desde disco

chroma_db = Chroma(persist_directory='save_db', embedding_function=vectorizador)

Podemos recuperar los documentos a través de una búsqueda por similitud con el método `similarity_search` de Chroma, pasándole como argumentos la consulta que hacemos y el número de documentos relevantes que queremos extraer de la base de datos para ser usados después como contexto.

In [ ]:
# búsqueda por similitud con el retorno de los 5 documentos más relevantes

documentos = chroma_db.similarity_search('Derivados de activos no corrientes 2022', k=5)

documentos[0]

In [ ]:
len(documentos)

También podemos crear un objeto recuperador para luego usarlo en la cadena de LangChain. Por defecto, el tipo de búsqueda que realiza el recuperador, `search_type`, es por similitud y devuelve los más relevantes según ella. Podemos usar también la similitud con un umbral para recuperar los documentos que sobrepasen cierto nivel de similitud con nuestra consulta. El recuperador dispone además de un algoritmo llamado `MMR (maximal marginal relevance)`. El algoritmo de relevancia marginal máxima selecciona documentos basándose en una combinación de qué documentos son más similares a las consultas, al mismo tiempo que optimiza la diversidad. Lo hace encontrando los ejemplos con embeddings que tienen la mayor similitud coseno con las entradas, y luego los va añadiendo iterativamente mientras les aplica una penalización por cercanía a los ejemplos ya seleccionados.

Usaremos el algoritmo MMR para que devuelva 20 documentos. El parámetro `lambda_mult` se refiere a la diversidad de los resultados devueltos por MMR siendo 1 para diversidad mínima y 0 para máxima. Por defecto es 0.5. Le pediremos un poco más de diversidad en su respuesta.

In [ ]:
recuperador = chroma_db.as_retriever(search_type='mmr', 
                                     search_kwargs={'k': 20, 'lambda_mult': 0.25})

In [ ]:
recuperador

## 7. Plantillas de instrucciones ([Prompt Templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/))

![prompts](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/prompts.png)

Las plantillas de instrucciones son recetas predefinidas para generar instrucciones para modelos de lenguaje. Una plantilla puede incluir instrucciones, contexto y preguntas específicas apropiadas para una tarea dada. LangChain proporciona herramientas para crear y trabajar con plantillas de instrucciones y además se esfuerza por crear plantillas agnósticas al modelo para facilitar la reutilización de plantillas existentes en diferentes modelos de lenguaje. Esta clase permite a los desarrolladores estructurar y manipular cómo se formulan las solicitudes (o "prompts") al modelo de lenguaje. Esto facilita una interacción más efectiva y controlada con el modelo, optimizando la generación de texto para casos de uso específicos.

Las funcionalidades pricipales son estas: 

1. Estructuración de Prompts: un template ayuda a organizar y estructurar los prompts que se envían a los modelos de lenguaje. Esto incluye la incorporación de datos específicos en una plantilla predefinida, lo que asegura que la información relevante se presente al modelo de manera coherente cada vez.

2. Reutilización: permite la reutilización de patrones de prompt comunes en diferentes partes de una aplicación, asegurando consistencia y reduciendo la duplicidad de código.

3. Personalización: se pueden definir múltiples plantillas para distintos tipos de tareas o estilos de interacción, lo que permite una personalización flexible según el contexto o las necesidades del usuario.

4. Integración con Datos: un template se utiliza para integrar dinámicamente datos en tiempo real dentro de los prompts, lo que es crucial para aplicaciones que dependen de información actualizada o específica del contexto para generar respuestas adecuadas.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# plantilla de texto con un contexto y una pregunta
plantilla = '''
            Answer the question based on the context below. If you can't 
            answer the question, reply "I don't know".

            Context: {context}

            Question: {question}
            
            Don´t response with the prompt. Translate the answer to Spanish.
            '''


# carga de la plantilla en el prompt
prompt = ChatPromptTemplate.from_template(plantilla)

prompt

## 8. Modelo LLM 

Para usar los modelos de [HuggingFace](https://huggingface.co/models), vamos a necesitar el [token](https://huggingface.co/settings/tokens) que nos proporciona la plataforma. Por supuesto, necesitamos crearnos una cuenta en HuggingFace. El token lo guardamos en un archivo `.env` para cargarlo con la librería dotenv y usarlo como variable de entorno. Dicho archivo se añade al `.gitignore` para que nadie pueda verlo si subimos el código a GitHub por ejemplo.


In [ ]:
from dotenv import load_dotenv      # carga variables de entorno 

load_dotenv()


# importamos el token

HUGGINGFACE_TOKEN = os.getenv('HUGGING_FACE_TOKEN')

![modelo](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/modelo.png)

Vamos a usar el modelo [zephyr](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), una versión del modelo mistralai/Mistral-7B-v0.1 que fue entrenado con una mezcla de conjuntos de datos sintéticos y públicamente disponibles utilizando optimización de preferencias directas (Direct Preference Optimization - [DPO](https://arxiv.org/abs/2305.18290)). Zephyr es un modelo de generación de texto con aproximadamente 7 mil millones de parámetros similar en rendimiento a GPT-3.5-turbo. 

Cargaremos el modelo desde el hub de HuggingFace. El parámetro `max_new_tokens` especifica el número máximo de tokens nuevos que el modelo puede generar en una sola invocación de su función de predicción o generación de texto. Usaremos 512 tokens máximos. Cuando un modelo genera texto, selecciona el siguiente token basándose en una distribución de probabilidad de todos los tokens posibles. El parámetro `top_k` restringe esta selección a los tokens más probables, donde k es un número entero definido por el usuario. Por ejemplo, si top_k está configurado en 30, el modelo solo considerará los 30 tokens más probables como candidatos para ser el siguiente token en la secuencia. Esto limita la elección a un subconjunto de posibilidades más probable y puede ayudar a que el texto generado sea más coherente y menos propenso a respuestas aleatorias o incoherencias. El parámetro `temperature` en un modelo es una configuración que afecta la aleatoriedad de las respuestas generadas por el modelo. Este parámetro ayuda a controlar cómo de conservador o aventurero será el modelo al seleccionar palabras durante la generación de texto. Una temperatura baja, 0.5 o menos, hace que el modelo sea más determinista, tendiendo a seleccionar los tokens más probables. Esto resulta en respuestas más coherentes y predecibles, pero potencialmente menos creativas o variadas. Una temperatura alta, 1.0 o más, hace que el modelo sea menos determinista y más propenso a tomar riesgos, seleccionando tokens menos probables. Esto puede generar respuestas más diversas y creativas, pero también puede aumentar la probabilidad de errores o respuestas incoherentes. Usaremos una temperatura de 0.1. El parámetro `repetition_penalty` en un modelo se utiliza para desalentar la repetición de palabras o frases en la generación de texto. Este ajuste ayuda a aumentar la variedad y la coherencia en las respuestas generadas, haciéndolas más agradables y naturales. Si $repetitionpenalty > 1$, la probabilidad de tokens repetidos disminuye, lo que desincentiva la repetición. Cuanto mayor sea el valor, más fuerte será el efecto en desalentar la repetición. Si $repetitionpenalty < 1$, la probabilidad de tokens repetidos aumenta, lo que podría no ser deseable pero puede ser útil en contextos específicos donde la repetición es preferida. Usaremos 1.03 como repetition_penalty.



In [ ]:
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace



llm = HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta',
                     task='text-generation',
                     huggingfacehub_api_token=HUGGINGFACE_TOKEN,
                     
                     model_kwargs={'max_new_tokens': 512,
                                   'top_k': 30,
                                   'temperature': 0.1,
                                   'repetition_penalty': 1.03})


modelo = ChatHuggingFace(llm=llm)

In [ ]:
# realizamos una prueba del modelo para ver que funciona

modelo.invoke('Capital de España')

## 9. Construcción de la cadena

Una cadena de LangChain es una secuencia configurada de componentes o pasos que se encadenan para realizar tareas complejas de procesamiento del lenguaje. LangChain es una herramienta diseñada para facilitar la construcción y el despliegue de aplicaciones de lenguaje, integrando modelos de lenguaje avanzados y otras funcionalidades en flujos de trabajo coherentes y efectivos. El propósito de una cadena de LangChain es permitir que cada paso del proceso contribuya de manera efectiva al resultado final. Una cadena de LangChain típicamente incluirá varios componentes clave:

+ Recuperadores (Retrievers): Componentes que buscan y recuperan información relevante de una base de datos o de un conjunto de documentos. Estos son esenciales para proporcionar contexto o datos específicos que el modelo de lenguaje necesita para generar respuestas informadas.

+ Modelos de Lenguaje: El núcleo de una cadena de LangChain, donde se utilizan modelos avanzados como GPT-3 para generar texto, resolver preguntas, o realizar otras tareas de NLP basadas en la información y contextos proporcionados por otros componentes de la cadena.

+ Post-procesadores: Componentes que toman la salida del modelo de lenguaje y la refinan, por ejemplo, corrigiendo errores, formateando la respuesta, o aplicando filtros adicionales de contenido.



Vamos a construir la cadena de LangChain. El proceso será:

1. Escribir la consulta.
2. Recuperar los documentos relevantes de la base de datos para generar el contexto.
3. Crear el prompt desde la plantilla con la consulta y el contexto.
4. Introducir al modelo el prompt.
5. Transformar la respuesta del modelo.


![chain](https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/chain.png)

Vamos a construir primero una cadena simple con la recuperación por similitud.

In [ ]:
consulta = '¿Cuál es el Balance de Situación Financiera Consolidado al 31 de diciembre de 2022?'


# cadena con la plantilla de prompt y el modelo, con LangChain Expression Language (LCEL)
cadena = prompt | modelo


# invocamos a la cadena con el contexto traido de la base de datos y la consulta 
respuesta = cadena.invoke({'context': chroma_db.similarity_search(consulta), 
                           'question': consulta})


# transformamos la respuesta del modelo 
respuesta.content.split('<|assistant|>')[1].strip()

RunnablePassthrough permite pasar las entradas sin cambios. De esta manera le pasamos la consulta tanto al recuperador como al modelo. Al recuperador de la base de datos le vamos a pedir que nos devuelva los 5 documentos más relevantes.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
consulta = '¿Cuál es el Balance de Situación Financiera Consolidado al 31 de diciembre de 2022?'


# recuperamos los 5 documentos más relevantes de la base de datos
recuperador = chroma_db.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'lambda_mult': 0.25})


# cadena con el recuperador, la plantilla de prompt y el modelo
cadena = {'context': recuperador, 'question': RunnablePassthrough()} | prompt | modelo


# respuesta de la cadena
respuesta = cadena.invoke(consulta)


# transformamos la respuesta del modelo
respuesta.content.split('<|assistant|>')[1].strip()

Así, ya tenemos disponible la recuperación de datos desde los PDFs según la consulta realizada al modelo LLM.

## 10. Resumen paso a paso

In [ ]:
%%time

# librerias
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace
from langchain_core.runnables import RunnablePassthrough

import os
from dotenv import load_dotenv
load_dotenv()

# importamos el token de huggingface
HUGGINGFACE_TOKEN = os.getenv('HUGGING_FACE_TOKEN')



# carga de todos los archivos PDF y realiza los chunks
loader = PyPDFDirectoryLoader('pdfs/')
documentos = loader.load_and_split()


# modelo embedding
vectorizador = HuggingFaceEmbeddings(model_name='sentence-transformers/all-roberta-large-v1')


# guardado en disco, no sería necesario, debería de hacerse aparte
chroma_db = Chroma.from_documents(documentos,                    # documentos de texto
                                  vectorizador,                  # modelo de embedding
                                  persist_directory="save_db"    # ruta de guardado
                                 )


# carga desde disco
chroma_db = Chroma(persist_directory='save_db', embedding_function=vectorizador)



# plantilla de texto con un contexto y una pregunta
plantilla = '''
            Answer the question based on the context below. If you can't 
            answer the question, reply "I don't know".

            Context: {context}

            Question: {question}
            
            Don´t response with the prompt. Translate the answer to Spanish.
            '''


# carga de la plantilla en el prompt
prompt = ChatPromptTemplate.from_template(plantilla)



# modelo de huggingface
llm = HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta',
                     task='text-generation',
                     huggingfacehub_api_token=HUGGINGFACE_TOKEN,
                     
                     model_kwargs={'max_new_tokens': 512,
                                   'top_k': 30,
                                   'temperature': 0.1,
                                   'repetition_penalty': 1.03})


modelo = ChatHuggingFace(llm=llm)


# consulta que hacemos
consulta = '¿Cuál es el Balance de Situación Financiera Consolidado al 31 de diciembre de 2022?'


# recuperamos los 5 documentos más relevantes de la base de datos
recuperador = chroma_db.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 0.25})


# cadena con el recuperador, la plantilla de prompt y el modelo
cadena = {'context': recuperador, 'question': RunnablePassthrough()} | prompt | modelo


# respuesta de la cadena
respuesta = cadena.invoke(consulta)


# transformamos la respuesta del modelo
respuesta.content.split('<|assistant|>')[1].strip()